# Extended Kalman Filter DSSE
The EKF method has two-steps i.e., prediction step and an update step. Note that the voltage magnitudes and voltage angles are the states to be estimated. Therefore, the number of states is twice as many compared to the total number of nodes in the system.

In [1]:
import os
import json
import math
from oedisi.types.data_types import MeasurementArray, AdmittanceMatrix, Topology
import pyarrow.feather as feather
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import plotly.express as px
ROOT = os.getcwd()
OUTPUTS = "outputs"

## Run OEDISI

In [2]:
SCENARIO_DIR, SCENARIOS, _ = next(os.walk(f'{ROOT}/scenario/'))

print(SCENARIO_DIR)
for idx, scenario in enumerate(SCENARIOS):
    print(idx, scenario)

/home/gray/git/oedisi_dopf/scenario/
0 medium
1 medium_extreme
2 medium_high
3 small
4 ekf_small_medium
5 medium_medium
6 omoo_medium_extreme
7 epri_j1
8 omoo_medium
9 medium_low
10 ekf_ieee123
11 ieee123
12 lest_small
13 small_medium
14 omoo
15 omoo_small
16 small_low
17 small_high
18 lest_test
19 ieee123_update
20 lest_ieee123
21 small_extreme
22 large
23 ekf_medium_medium


In [3]:
index = 10
SCENARIO = SCENARIOS[index]
path = f"{SCENARIO_DIR}{SCENARIO}"

In [4]:
# os.system(f"oedisi build --system {path}/system.json --component-dict {path}/components.json --target-directory build_{SCENARIO}") 

In [5]:
# os.system("pkill -9 helics_broker")
# os.system(f"oedisi run --runner build_{SCENARIO}/system_runner.json")

## Load Outputs

In [6]:
directory = f"{OUTPUTS}/{SCENARIO}"
v_real = feather.read_feather(
    os.path.join(directory, "voltage_real.feather")
)
v_imag = feather.read_feather(
    os.path.join(directory, "voltage_imag.feather")
)
true_voltages = v_real.drop("time", axis=1) + 1j * v_imag.drop("time", axis=1)
true_voltages['time'] = pd.to_datetime(v_real['time'],format='%Y-%m-%d %H:%M:%S')
true_voltages.set_index('time', inplace=True)

v_mag = feather.read_feather(
    os.path.join(directory, "voltage_mag.feather")
)
v_angle = feather.read_feather(
    os.path.join(directory, "voltage_angle.feather")
)
v_rad = np.radians(v_angle.drop("time", axis=1).to_numpy())
estimated_voltages = v_mag.drop("time", axis=1) * np.exp(1j * v_rad)
estimated_voltages['time'] = pd.to_datetime(v_angle['time'],format='%Y-%m-%d %H:%M:%S')
estimated_voltages.set_index('time', inplace=True)

with open(os.path.join(directory, "topology.json")) as f:
    topology = Topology.parse_obj(json.load(f))
    base_voltage_df = pd.DataFrame(
        {
            "id": topology.base_voltage_magnitudes.ids,
            "value": topology.base_voltage_magnitudes.values,
        }	
    )
    base_voltage_df.set_index("id", inplace=True)
    base_voltages = base_voltage_df["value"]
    

## 

In [10]:
df_error = (true_voltages.abs()/base_voltages - estimated_voltages.abs()/base_voltages).abs()
df_error = (df_error
      .dropna()
      .reset_index()
      .melt(id_vars=["time"], var_name="node", value_name="absolute")
      .drop_duplicates())

df_pct_error = ((true_voltages.abs()/base_voltages - estimated_voltages.abs()/base_voltages)/(true_voltages.abs()/base_voltages)).abs()
df_pct_error = (df_pct_error
                .dropna()
                .reset_index()
                .melt(id_vars=["time"], var_name="node", value_name="percent")
                .drop_duplicates())

df_error["percent"] = df_pct_error.percent
del df_pct_error
df_error["phase"] = df_error.node.str.split(".").str[1]
df_error.node = df_error.node.str.split(".").str[0]

In [11]:

mean_per_time = df_error.loc[:, ["time", "phase", "absolute", "percent"]].groupby(by=["time", "phase"]).mean().reset_index()
px.scatter(mean_per_time, x="time", y=["absolute", "percent"], facet_col="phase", title="Mean Error vs. Time").show()
mean_per_node = df_error.groupby(["node", "phase"]).mean().reset_index()
px.scatter(mean_per_node, x="node", y=["absolute", "percent"], facet_col="phase", title="Mean Error vs. Node").show()
print(df_error.loc[:, ["absolute", "percent"]].mean())

absolute    0.01484
percent     0.01435
dtype: float64


In [8]:
df_error_worst = (true_voltages.abs()/base_voltages - 1).abs()
print("Mean absolute error if all estimates=1pu: ", df_error_worst.mean().mean())

Mean absolute error if all estimates=1pu:  0.023297997840627603


In [9]:
time='12:30'
def compare_voltages(true_voltages_df, estimated_voltages_df, time: str="12:30"):
    out_dir = Path(f"{OUTPUTS}/{SCENARIO}")
    true = true_voltages_df.at_time(time)/base_voltages
    estimated = estimated_voltages_df.at_time(time)/base_voltages
    # plot_errors(err_table).savefig(f"{directory}/errors_noon.png")
    # print(err_table.head())
    true = true.T
    true.columns.name = None
    true["true"] = true.iloc[:, 0]
    true = true.loc[:, ["true"]].copy()
    estimated = estimated.T
    estimated.columns.name = None
    estimated["estimate"] = estimated.iloc[:, 0]
    estimated = estimated.loc[:, ["estimate"]].copy()
    df = pd.concat([true, estimated], axis=1)
    df["id"] = df.index
    df = df.melt(id_vars=["id"])
    # true.index = true.id
    # print(true.keys())
    df["node"] = df.id.str.split(".").str[0]
    df["phase"] = df.id.str.split(".").str[1]
    df.head()
    df["mag"] = df.value.abs()
    df["angle"] = df.value.apply(np.angle)
    df.head()
    
    fig = px.scatter(df, x="node", y="mag", facet_col="phase", color="variable", symbol="variable", title=f"{SCENARIO} at {time}")
    time_id = time.replace(":", "_")
    fig.write_html(out_dir / f"voltages_at_{time_id}.html")
    fig.show()


def voltages_differences(true_voltages_df, estimated_voltages_df, time: str="12:30"):
    out_dir = Path(f"{OUTPUTS}/{SCENARIO}")
    true = true_voltages_df.at_time(time)/base_voltages
    estimated = estimated_voltages_df.at_time(time)/base_voltages
    # plot_errors(err_table).savefig(f"{directory}/errors_noon.png")
    # print(err_table.head())
    true = true.T
    true.columns.name = None
    true["true"] = true.iloc[:, 0]
    true = true.loc[:, ["true"]].copy()
    estimated = estimated.T
    estimated.columns.name = None
    estimated["estimate"] = estimated.iloc[:, 0]
    estimated = estimated.loc[:, ["estimate"]].copy()
    df = pd.concat([true, estimated], axis=1)
    df["error"] = df.true.apply(np.abs) - df.estimate.apply(np.abs)
    df["id"] = df.index
    df = df.loc[:, ["id", "error"]]
    df["node"] = df.id.str.split(".").str[0]
    df["phase"] = df.id.str.split(".").str[1]
    df.head()
    fig = px.scatter(df, x="node", y="error", facet_col="phase", title=f"{SCENARIO} voltage errors at {time} (actual-estimate)")
    time_id = time.replace(":", "_")
    fig.write_html(out_dir / f"voltage_errors_at_{time_id}.html")
    fig.show()

def voltage_angle_errors(true_voltages_df, estimated_voltages_df, time: str="12:30"):
    out_dir = Path(f"{OUTPUTS}/{SCENARIO}")
    true = true_voltages_df.at_time(time)/base_voltages
    estimated = estimated_voltages_df.at_time(time)/base_voltages
    # plot_errors(err_table).savefig(f"{directory}/errors_noon.png")
    # print(err_table.head())
    true = true.T
    true.columns.name = None
    true["true"] = true.iloc[:, 0]
    true = true.loc[:, ["true"]].copy()
    estimated = estimated.T
    estimated.columns.name = None
    estimated["estimate"] = estimated.iloc[:, 0]
    estimated = estimated.loc[:, ["estimate"]].copy()
    df = pd.concat([true, estimated], axis=1)
    df["id"] = df.index
    df = df.melt(id_vars=["id"])
    # true.index = true.id
    # print(true.keys())
    df["node"] = df.id.str.split(".").str[0]
    df["phase"] = df.id.str.split(".").str[1]
    df.head()
    df["mag"] = df.value.abs()
    df["angle"] = df.value.apply(np.angle)*180/np.pi
    df.loc[df.phase == "2", "angle"] = df.loc[df.phase == "2", "angle"] + 120
    df.loc[df.phase == "3", "angle"] = df.loc[df.phase == "3", "angle"] - 120
    df.head()
    
    fig = px.scatter(df, x="node", y="angle", facet_col="phase", color="variable", symbol="variable", title=f"{SCENARIO} Voltage Angles (Degrees) At {time} (relative to nominal phase angle)")
    time_id = time.replace(":", "_")
    fig.write_html(out_dir / f"voltages_at_{time_id}.html")
    fig.show()

compare_voltages(true_voltages, estimated_voltages, "7:30")
compare_voltages(true_voltages, estimated_voltages, "12:30")
compare_voltages(true_voltages, estimated_voltages, "17:30")
voltages_differences(true_voltages, estimated_voltages, "7:30")
voltages_differences(true_voltages, estimated_voltages, "12:30")
voltages_differences(true_voltages, estimated_voltages, "17:30")
voltage_angle_errors(true_voltages, estimated_voltages, "7:30")
